In [ ]:
import os
import re
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as compare_ssim
from scipy.stats import pearsonr
from skimage.color import rgb2gray
import csv
from PIL import Image 
import re

def __normalize_image_to_01(np_img):
    min_val = np.min(np_img)
    max_val = np.max(np_img)
    
    if max_val - min_val != 0:
        np_img = (np_img - min_val) / (max_val - min_val)
    else:

        np_img = np.zeros(np_img.shape, dtype=np.float32)
    #np_img = Image.fromarray((np_img * 255).astype(np.uint8))
    
    return np_img

def extract_id(filename):
    """
    从文件名中提取ID。
    
    参数:
        filename (str): 文件名。
    
    返回:
        str: 提取的ID，如果没有找到则返回None。
    """
    match = re.search(r"(\d+)", filename)
    return match.group(1) if match else None

def calculate_metrics(image_path_1, image_path_2):

    image1 = np.array(Image.open(image_path_1))
    image1 = __normalize_image_to_01(image1)
    image2 = np.array(Image.open(image_path_2))
    image2 = __normalize_image_to_01(image2)
    
    if len(image1.shape) > 2:
        image1_gray = rgb2gray(image1)
        image2_gray = rgb2gray(image2)
    else:
        image1_gray = image1
        image2_gray = image2
    
    ssim = np.round(compare_ssim(image1_gray, image2_gray, data_range=image1_gray.max() - image1_gray.min()), 4)

    pcc = np.round(pearsonr(image1_gray.flatten(), image2_gray.flatten())[0], 4)
    
    mae = np.round(np.mean(np.abs(image1_gray - image2_gray)), 4)
    
    ecd = np.round(np.sqrt(np.sum((image1_gray - image2_gray) ** 2)), 4)
    
    return ssim, pcc, mae, ecd

    
def compare_folders(folder1, folder2, output_csv):
    files1 = {extract_id(f): f for f in os.listdir(folder1) if extract_id(f)}
    files2 = {extract_id(f): f for f in os.listdir(folder2) if extract_id(f)}

    metrics = []

    for id, file1 in files1.items():
        if id in files2:
            path1 = os.path.join(folder1, file1)
            path2 = os.path.join(folder2, files2[id])
            
            ssim, pcc, mae, ecd = calculate_metrics(path1, path2)
            metrics.append([file1, files2[id], ssim, pcc, mae, ecd])
        else:
            print(f"No corresponding file for ID {id} found in folder B.")
    

    formatted_metrics = []
    for metric in metrics:
        formatted_metric = [
            metric[0],  # ImageA
            metric[1],  # ImageB
            f"{metric[2]:.4f}",  # SSIM
            f"{metric[3]:.4f}",  # PCC
            f"{metric[4]:.4f}",  # MAE
            f"{metric[5]:.4f}",  # ECD
        ]
        formatted_metrics.append(formatted_metric)

    output_csv = 'Tubulin_metrics.csv'
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ImageA', 'ImageB', 'SSIM', 'PCC', 'MAE', 'ECD'])
        writer.writerows(formatted_metrics)


folder1 = './datasets/fold_data/Actin_fold_0/testB/'
folder2 = './results/Actin_fold_0/test_latest/images/'
output_csv = './metrics.csv'

compare_folders(folder1, folder2, output_csv)


In [1]:
import os
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim
from scipy.stats import pearsonr
from skimage.color import rgb2gray
import csv
from PIL import Image 
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def __normalize_image_to_01(np_img):
    """
    将图像标准化到 [0, 1] 范围内。
    """
    min_val = np.min(np_img)
    max_val = np.max(np_img)
    
    if max_val - min_val != 0:
        np_img = (np_img - min_val) / (max_val - min_val)
    else:
        np_img = np.zeros(np_img.shape, dtype=np.float32)
    
    return np_img


def calculate_metrics(image_path_1, image_path_2):
    """
    计算两个图像之间的相似性度量。
    """
    #image1 = np.array(Image.open(image_path_1))
    #print('image1', image1.shape)
    #image1 = __normalize_image_to_01(image1)
    #image1_resized = Image.fromarray(image1).resize((512, 512))
    #image1_resized = np.array(image1_resized)
#
    #image2 = np.array(Image.open(image_path_2))
    #print('image2', image2.shape)
    #image2 = __normalize_image_to_01(image2)
    #image2_resized = Image.fromarray(image2).resize((512, 512))
    #image2_resized = np.array(image2_resized)
    
    image1 = np.array(Image.open(image_path_1))
    print('image1', image1.shape)
    image1_resized = __normalize_image_to_01(image1)

    image2 = np.array(Image.open(image_path_2))
    print('image2', image2.shape)
    image2 = __normalize_image_to_01(image2)

    image2_resized = Image.fromarray(image2).resize(image1.shape[::-1])  
    image2_resized = np.array(image2_resized)
    print('image2_resized', image2_resized.shape)
    
    if len(image1_resized.shape) > 2:
        image1_gray = rgb2gray(image1_resized)
        image2_gray = rgb2gray(image2_resized)
    else:
        image1_gray = image1_resized
        image2_gray = image2_resized
    
    ssim = np.round(compare_ssim(image1_gray, image2_gray, data_range=image1_gray.max() - image1_gray.min()), 4)

    pcc = np.round(pearsonr(image1_gray.flatten(), image2_gray.flatten())[0], 4)
    
    mae = np.round(np.mean(np.abs(image1_gray - image2_gray)), 4)
    
    #ecd = np.round(np.sqrt(np.sum((image1_gray - image2_gray) ** 2)), 4)
    
    e_dist = euclidean_distances(image1_gray.ravel().reshape(1, -1),
                                     image2_gray.ravel().reshape(1, -1))[0, 0]
    c_dist = np.abs(cosine_distances(image1_gray.ravel().reshape(1, -1),
                                         image2_gray.ravel().reshape(1, -1))[0, 0])
    
    
    
    return ssim, pcc, mae, e_dist , c_dist


    
def compare_folders(folder1, folder2, output_csv):
    """
    比较两个文件夹中的图像，并计算相似性度量。
    """
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)
    #print(files2)
    metrics = []

    for file1 in files1:
        if file1.endswith('.ome.tiff'):
            for file2 in files2:
                if file2.endswith('.ome_fake_B.png') and file1.split('.')[0] == file2.split('.')[0]:
                    path1 = os.path.join(folder1, file1)
                    path2 = os.path.join(folder2, file2)
                    ssim, pcc, mae, e_dist, c_dist = calculate_metrics(path1, path2)
                    metrics.append([file1, file2, ssim, pcc, mae, e_dist, c_dist])
                    break  

    formatted_metrics = []
    for metric in metrics:
        formatted_metric = [
            metric[0],  # ImageA
            metric[1],  # ImageB
            f"{metric[2]:.4f}",  # SSIM
            f"{metric[3]:.4f}",  # PCC
            f"{metric[4]:.4f}",  # MAE
            f"{metric[5]:.4f}",  # ECD
            f"{metric[6]:.4f}",  # ECD
        ]
        formatted_metrics.append(formatted_metric)

    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ImageA', 'ImageB', 'SSIM', 'PCC', 'MAE', 'E_dist', 'C_dist'])
        writer.writerows(formatted_metrics)


folder1 = './datasets/fold_data/Mitochondria_fold_3/testB/'
folder2 = './results/Mitochondria_fold_3/test_latest/images/'
output_csv = './metrics4.csv'

compare_folders(folder1, folder2, output_csv)


image1 (966, 1296)
image2 (1024, 1024)
image2_resized (966, 1296)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (2048, 2048)
image2 (1024, 1024)
image2_resized (2048, 2048)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (2048, 2048)
image2 (1024, 1024)
image2_resized (2048, 2048)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (2048, 2048)
image2 (1024, 1024)
image2_resized (2048, 2048)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (2048, 2048)
image2 (1024, 1024)
image2_resized (2048, 2048)
image1 (1200, 1200)
image2 (1024, 1024)
image2_resized (1200, 1200)
image1 (512, 512)
image2 (1024, 1024)
image2_resized (512, 512)
image1 (512, 512)
